### Import relevant libraries

In [10]:
from semopy import Model, Optimizer
import matplotlib.pyplot as plt
from scipy.stats import f_oneway
import pandas as pd
import numpy as np
import seaborn as sns

### Import the dataset

In [11]:
df = pd.read_csv("Welzijnsmonitor2025_scaled_normalised_UTF8.csv", sep=";")

### Creating the model

In [ ]:
model_desc = """
# Measurement model

Bevlogenheid_en_Burnout_1 =~ Bevl_1 + Bevl_2 + Bevl_3
Gedrag_1 =~ Cogn_Eng1_5 + Cogn_Eng1_6 + Cogn_Eng2_2 + Cogn_Eng2_3 + Cogn_Eng2_8
Motivatie =~ Mot_Stress_1 + Mot_Stress_4
Veerkracht_1 =~ Veer_1 + Veer_5
Welbevinden =~ Bevlogenheid_en_Burnout_1 + Gedrag_1 + Motivatie + Veerkracht_1

Bekendheid_Interventies =~ Bekendgebruik_1 + Bekendgebruik_2 + Bekendgebruik_3 + Bekendgebruik_4 + Bekendgebruik_5 + Bekendgebruik_6 + Bekendgebruik_7

Achtergrondkenmerken_1 =~ Betrok_Ouders

Gedrag_2 =~ Cogn_Eng1_1 + Cogn_Eng1_2 + Cogn_Eng1_3 + Cogn_Eng1_4 + Cogn_Eng2_1 + Cogn_Eng2_4 + Cogn_Eng2_5 + Cogn_Eng2_6 + Cogn_Eng2_7

Extra_Engagement =~ Cogregiedocent_1 + Cogregiedocent_2 + Cogregiedocent_3 +  Cogregiedocent_4 + Cogregiedocent_5 + Cogregiedocent_6 + Extr_eng2_1 + Extr_eng2_2 + Extr_eng2_3

Hulpbehoefte =~ Hulp_1 + Hulp_2 + Hulp_3
Participatie =~ Partici1_1 + Partici1_2 + Partici1_3 + Partici1_4

Energiebronnen =~ Bekendheid_Interventies + Achtergrondkenmerken_1 + Gedrag_2 + Extra_Engagement + Hulpbehoefte + Participatie

Negatieve_Uitkomsten =~ StopInt + Vertr

Studieprestaties =~ Cijfer_huidig_1 + StPunt_beh
Achtergrondkenmerken_2 =~ Stopint2
Positieve_Uitkomsten =~ Studieprestaties + Achtergrondkenmerken_2

Leefstijl =~ Cantrill_1 + Leefst + Q297_1

Stress_1 =~ Onnodige_stress_1 + Onnodige_stress_2 + Onnodige_stress_3 + Onnodige_stress_4 + Onnodige_stress_5 + Onnodige_stress_6 + Onnodige_stress_7 + Onnodige_stress_8 + Onnodige_stress_9 + Onnodige_stress_10

Prestatiedruk =~ Pres_1 + Pres_2 + Pres_12
Achtergrondkenmerken_3 =~ Werk_1

Zorgen =~ Zorg_1 + Zorg_2 + Zorg_3 + Zorg_4 + Zorg_5 + Zorg_6

Stressoren =~ Leefstijl + Stress_1 + Prestatiedruk + Achtergrondkenmerken_3 + Zorgen

Bevlogenheid_en_Burnout_2 =~ Burn1_1 + Burn1_2 + Burn1_3 + Burn1_4
Depressie =~ Depr_1 + Depr_2 + Depr_3 + Depr_4 + Depr_5 + Depr_6 + Depr_7 + Depr_8

Stress_2 =~ Mot_Stress_2
Veerkracht_2 =~ Veer_2 + Veer_3 + Veer_4 + Veer_6

Stressreactie =~ Bevlogenheid_en_Burnout_2 + Depressie + Stress_2 + Veerkracht_2
"""

In [14]:
model = Model(model_desc)
model.fit(df)
latent_scores = model.predict_factors(df)

# Export the model with the new latent variables

In [16]:
latent_scores = model.predict_factors(df)

latent_scores.to_csv("latent_variable_scores.csv", index=False)

print("Latent variable scores extracted:")
print(latent_scores.head())

Latent variable scores extracted:
   Achtergrondkenmerken_1  Achtergrondkenmerken_2  Achtergrondkenmerken_3  \
0                0.516109                0.000925               -0.129402   
1                0.369231                0.015490               -0.320264   
2                0.506303                0.023396               -0.156456   
3               -0.161854                0.004191               -0.315727   
4                0.381887                0.008432               -0.302764   

   Bekendheid_Interventies  Bevlogenheid_en_Burnout_1  \
0                -0.253010                   0.257458   
1                -0.258810                   0.054494   
2                -0.253398                   0.433510   
3                -0.259518                  -0.130457   
4                -0.232772                  -0.118808   

   Bevlogenheid_en_Burnout_2  Depressie  Energiebronnen  Extra_Engagement  \
0                  -0.131377  -0.239755        0.011190          0.003618   
1     

In [ ]:
# regressions
# Stressoren ~ Energiebronnen
# Energiebronnen ~ Stressoren

# Stressreacties ~ Stressoren + Energiebronnen + Welbevinden
# Welbevinden ~ Stressoren + Energiebronnen + Stressreacties

# Positieve_Uitkomsten ~ Welbevinden + Stressreacties
# Negatieve_Uitkomsten ~ Welbevinden + Stressreacties